In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LassoCV 
from sklearn.feature_selection import SelectFromModel 
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import LabelEncoder
import joblib
from sklearn.feature_selection import mutual_info_classif
from sklearn.preprocessing import StandardScaler
import numpy as np

# Feature Selection

In [2]:
df=pd.read_csv('GDCdata\data_30_cancer.csv')
X = df.drop(columns = ["Unnamed: 0", "_primary_disease"])
y = df['_primary_disease'].values

In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.3, random_state=42)

## Information Gain


In [4]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

In [5]:
mutual_info = mutual_info_classif(X_train_scaled, y_train, n_jobs=-1)
mutual_info = pd.Series(mutual_info)
mutual_info.index = X_train.columns
mutual_info.sort_values(ascending=False)

4278     0.886643
14836    0.852830
11254    0.850518
4665     0.833722
8169     0.830028
           ...   
9969     0.000000
13423    0.000000
15940    0.000000
13419    0.000000
6852     0.000000
Length: 20530, dtype: float64

In [6]:
mean_ig = np.mean(mutual_info)
std_ig = np.std(mutual_info)
threshold = mean_ig + std_ig

In [7]:
selected_features = mutual_info[mutual_info.values > threshold]
X_train_selected = X_train[selected_features.index]
X_test_selected = X_test[selected_features.index]

In [8]:
X_test_selected.head()

,17,19,24,26,34,38,39,40,44,49,...,20464,20465,20472,20480,20485,20497,20505,20508,20510,20511
9185,7.6765,6.7131,8.2200,11.0616,2.3948,8.8880,1.8300,1.4349,2.9661,6.2471,...,0.0000,7.4209,8.1225,11.0148,14.3361,9.5204,5.0231,8.6700,10.6928,0.0000
3544,11.2073,7.0139,9.3820,9.2484,4.7020,6.7076,0.0000,0.3885,6.3891,9.4587,...,0.9464,9.6473,10.2841,9.3746,12.8122,9.6289,3.3604,11.3040,11.4745,5.6387
6274,11.4920,5.5021,9.9265,9.4504,2.1124,7.8443,0.0000,0.6360,6.4035,8.6274,...,1.0759,9.5540,11.6191,10.0427,14.8181,8.4049,0.0000,10.0860,10.0970,7.7803
5948,7.8140,6.7212,7.0457,7.9516,2.5865,6.7297,0.0000,2.2494,2.4277,10.5015,...,7.7856,11.1102,9.4666,10.6066,13.5944,7.3659,0.7011,9.8282,8.5026,8.8088
3141,12.0381,6.2966,7.3193,10.0199,1.5054,7.9806,0.4519,3.5443,2.4366,9.0100,...,7.5449,10.6540,8.5956,10.8656,13.9694,6.6324,0.0000,7.3922,9.3577,9.6364


# Lasso

In [9]:
def lasso_selection(X_train, X_test, y_train):
    lasso_cv = LassoCV(cv=15, verbose=2, max_iter=4500, n_jobs=5) 
    lasso_cv.fit(X_train, y_train)
    joblib.dump(lasso_cv, 'GDCdata/data_processed/lasso/lasso_model.pkl')

    sfm = SelectFromModel(lasso_cv, prefit=True, threshold=0.05) 
    X_train_selected = sfm.transform(X_train) 
    X_test_selected = sfm.transform(X_test) 
    return X_train_selected, X_test_selected

In [10]:
lb = LabelEncoder()
y_encoder = lb.fit_transform(y_train)

In [ ]:
X_train_selected_ls, X_test_selected_ls = lasso_selection(X_train_selected, X_test_selected, y_encoder)

[Parallel(n_jobs=5)]: Using backend ThreadingBackend with 5 concurrent workers.


Path: 000 out of 100Path: 000 out of 100

Path: 001 out of 100
Path: 001 out of 100
Path: 002 out of 100
Path: 002 out of 100
Path: 003 out of 100
Path: 003 out of 100
Path: 004 out of 100
Path: 004 out of 100
Path: 005 out of 100
Path: 005 out of 100
Path: 006 out of 100
Path: 006 out of 100
Path: 007 out of 100
Path: 000 out of 100
Path: 007 out of 100
Path: 008 out of 100
Path: 001 out of 100
Path: 000 out of 100
Path: 008 out of 100
Path: 009 out of 100
Path: 002 out of 100
Path: 001 out of 100
Path: 003 out of 100
Path: 010 out of 100
Path: 009 out of 100
Path: 002 out of 100
Path: 003 out of 100
Path: 004 out of 100
Path: 010 out of 100
Path: 011 out of 100
Path: 004 out of 100
Path: 005 out of 100
Path: 011 out of 100
Path: 012 out of 100
Path: 005 out of 100
Path: 006 out of 100
Path: 013 out of 100
Path: 012 out of 100
Path: 006 out of 100
Path: 007 out of 100
Path: 014 out of 100
Path: 000 out of 100
Path: 013 out of 100
Path: 007 out of 100
Path: 008 out of 100
Path: 001 out

d:\ancocda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:683: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 98.02601521277393, tolerance: 53.962280831689206
  model = cd_fast.enet_coordinate_descent_gram(


Path: 068 out of 100
Path: 067 out of 100
Path: 068 out of 100
Path: 068 out of 100
Path: 068 out of 100


d:\ancocda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:683: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 107.18093508893799, tolerance: 53.962280831689206
  model = cd_fast.enet_coordinate_descent_gram(


...

Path: 075 out of 100
Path: 075 out of 100


d:\ancocda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:683: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 120.63963507841981, tolerance: 53.962280831689206
  model = cd_fast.enet_coordinate_descent_gram(


Path: 075 out of 100


d:\ancocda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:683: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 88.24524131868384, tolerance: 53.55096543715847
  model = cd_fast.enet_coordinate_descent_gram(


Path: 075 out of 100
Path: 076 out of 100


In [ ]:
data_train = pd.DataFrame(X_train_selected_ls)
data_train['label'] = y_train
data_train.to_csv(r'GDCdata\data_processed\lasso\train\data_train_ls.csv')


In [ ]:
data_test = pd.DataFrame(X_test_selected_ls)
data_test['label'] = y_test
data_test.to_csv(r'GDCdata\data_processed\lasso\test\data_test_ls.csv')